In [1]:
import pandas as pd                    # dataframes
import numpy as np                     # any np operations
import matplotlib.pyplot as plt        # for plots
import seaborn as sns                  # plots
import warnings                        # supress deprecation or warning messages
%matplotlib inline                     
%load_ext autoreload
%autoreload 2
warnings.simplefilter("ignore")

In [2]:
dataset = pd.read_excel('./Dataset/2010 Federal STEM Education Inventory Data Set.xls', skiprows=1)
dataset.columns

Index(['Index Number', 'Investment Name', 'Agency', 'Subagency',
       'A) Brief Description', 'B) Year Established', 'C1) Funding FY2008',
       'C2) Funding FY2009', 'C3) Funding FY2010',
       'D) Mission-specific or General STEM?',
       ...
       'Unnamed: 246', 'Unnamed: 247',
       'Y3) Process or Implementation Evaluation Type', 'Unnamed: 249',
       'Unnamed: 250', 'Y4) Portfolio Evaluation/Review Type', 'Unnamed: 252',
       'Unnamed: 253', 'Y5) Expert Review Type', 'Unnamed: 255'],
      dtype='object', length=256)

In [3]:
def fill_funding(col, df):
    '''
        #### intution ###
        filling missing funding values before tagging growth rate
    '''
    if col is None or df is None:
        return None
    
    funding_rec = pd.DataFrame(df.groupby('Agency')[col].mean())
    print(funding_rec.head())
    company_details =[(x,df['Agency'][x]) for x in range(df.shape[0]) if not df[col][x]>=0]
    ## print(comany_details)
    ## tuple index, agency
    for x in company_details:
        print(x[0], x[1])
        df[col][x[0]] = funding_rec[col][x[1]]

In [4]:
### filling missing values for each year
fill_funding('C1) Funding FY2008',dataset)
fill_funding('C2) Funding FY2009',dataset)

             C1) Funding FY2008
Agency                         
Agriculture            4.587267
Commerce               3.115588
Defense                6.778314
Education             47.431857
Energy                 1.215943
52 NASA
55 NASA
56 NASA
59 NASA
69 NASA
70 NASA
72 NASA
73 NASA
79 NASA
85 NASA
86 NASA
88 NASA
92 NASA
98 Defense
113 Environmental Protection Agency
121 Energy
122 Energy
128 Energy
147 Education
218 Agriculture
239 Defense
250 Agriculture
             C2) Funding FY2009
Agency                         
Agriculture            4.365933
Commerce               3.471767
Defense                8.483045
Education             58.277018
Energy                 1.717495
56 NASA
72 NASA
73 NASA
85 NASA
98 Defense
128 Energy
147 Education
218 Agriculture
250 Agriculture


In [5]:
## let's tag for each investment
def growth_rate(past, cur):
    '''
        ### STAGE 1 FUNCTION ###
        for tagging investment names based on growth rate percentage
    '''
    if past:
        # check for past is zero
        if cur:
            return 1 if round(((cur-past)*100/past),2) >0 else 0
        else:
            return 0
    elif cur == 0 or cur is None:
        return 0
    else:
        # logically previously no funding but now we have, so positive
        return 1
    
### checking if adding a feature growth percentage can impact 
def add_growth_per(df):
    '''
        ### STAGE 1 FUNCTION ###
        Also to add extra column as growth_percentage
    '''
    if 'C2) Funding FY2009' in df.columns and 'C1) Funding FY2008' in df.columns:
        df['growth_percentage'] = ((df['C2) Funding FY2009']-df['C1) Funding FY2008'])*100/df['C1) Funding FY2008'])
        return df
    else:
        return None


In [6]:
## adding percentage growth rate
add_growth_per(dataset)
dataset.shape

(253, 257)

In [7]:
## time to tag Investment name
dataset['target'] = dataset.apply(lambda x: growth_rate(x['C1) Funding FY2008'],x['C2) Funding FY2009']), axis=1)
dataset.target.head()

0    1
1    0
2    0
3    1
4    1
Name: target, dtype: int64

In [9]:
labels = dataset.target
features = dataset.drop(['target'], axis=1)
#### STAGE 3 TASK 1 - split by 7:3
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.3,random_state=26, shuffle=True)

In [14]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train[['C1) Funding FY2008','C2) Funding FY2009','growth_percentage']].values, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [15]:
from sklearn.metrics import roc_auc_score
roc_auc_score(model.predict(X_test[['C1) Funding FY2008','C2) Funding FY2009','growth_percentage']].values), y_test)

1.0